In [1]:
!pip install opencv-python opencv-python-headless

In [2]:
import subprocess
def run_shell_command(command):
    """Helper function to run shell commands and handle errors."""
    result = subprocess.run(command, capture_output=True, text=True, shell=True)
    if result.returncode != 0:
        print(f"Error: {result.stderr}")
    return result.stdout

In [4]:
# run this command to start the container
command = "docker run -it --rm --privileged -e DISPLAY=$DISPLAY -v /tmp/.X11-unix/:/tmp/.X11-unix:ro stella_vslam-iridescense"

In [3]:
# get the container id
import subprocess
print("Getting the first Docker container ID...")
container_id = subprocess.check_output("docker ps -q", shell=True).decode().strip()
print(f"Container ID: {container_id}")

Getting the first Docker container ID...
Container ID: 6ab7becb1dfb


In [21]:
import ffmpeg

# Function to extract video metadata using ffmpeg
def extract_video_metadata(video_path):
    # Use ffmpeg.probe to retrieve metadata of the video
    probe = ffmpeg._probe.probe(video_path, v='error', select_streams='v:0', show_entries='stream=width,height,r_frame_rate')
    
    # Extract relevant metadata from the probe results
    video_stream = probe['streams'][0]
    
    # Extract FPS, width (cols), and height (rows)
    fps = eval(video_stream['r_frame_rate'])  # 'r_frame_rate' is in the form of "numerator/denominator"
    cols = video_stream['width']
    rows = video_stream['height']
    
    return {
        'fps': fps,
        'cols': cols,
        'rows': rows
    }

# Path to the video file
video_name = input("Please enter the video file path: ")

# Extract metadata
video_metadata = extract_video_metadata('./input/'+video_name)
print(video_metadata)


{'fps': 30.0, 'cols': 1280, 'rows': 720}


In [22]:
import yaml
yaml_content = f"""
# Equirectangular model for RICOH THETA S

#==============#
# Camera Model #
#==============#

Camera:
  name: "RICOH THETA S 960"
  setup: "monocular"
  model: "equirectangular"  # Ensure this matches your camera's model
  fps: {video_metadata['fps']}
  cols:  {video_metadata['cols']}
  rows:  {video_metadata['rows']}
  color_order: "RGB"
  # k1: 0.0
  # k2: 0.0
  # k3: 0.0
  # p1: 0.0
  # p2: 0.0
  # fx: 500.0                           # Focal length in the x-direction
  # fy: 500.0                           # Focal length in the y-direction
  # cx: 960.0                           # Principal point x-coordinate (center of the image)
  # cy: 540.0
  baseline: 0.1   

#================#
# ORB Parameters #
#================#

Feature.max_num_keypoints: 2000
Feature.scale_factor: 1.2
Feature.num_levels: 8
Feature.ini_fast_threshold: 20
Feature.min_fast_threshold: 7
Feature.mask_rectangles:
  - [0.0, 1.0, 0.0, 0.1]
  - [0.0, 1.0, 0.84, 1.0]
  - [0.0, 0.2, 0.7, 1.0]
  - [0.8, 1.0, 0.7, 1.0]
"""

# Write the content to a file
file_path = './input/config.yaml'

with open(file_path, 'w') as file:
    file.write(yaml_content)

print(f"YAML file has been created at {file_path}")


YAML file has been created at ./input/config.yaml


In [23]:
# copy video and config to container
print(f"Copying inputs to container {container_id}...")
subprocess.run(
    f"docker cp ./input {container_id}:/stella_vslam_examples/build/input",
    shell=True
)

Copying inputs to container 6ab7becb1dfb...


CompletedProcess(args='docker cp ./input 6ab7becb1dfb:/stella_vslam_examples/build/input', returncode=0)

In [20]:
# repair the container
subprocess.run(
    f"docker exec {container_id} mkdir output",
    shell=True
)
subprocess.run(
    f'docker exec {container_id} curl -sL "https://github.com/stella-cv/FBoW_orb_vocab/raw/main/orb_vocab.fbow" -o orb_vocab.fbow',
    shell=True
)


CompletedProcess(args='docker exec 6ab7becb1dfb curl -sL "https://github.com/stella-cv/FBoW_orb_vocab/raw/main/orb_vocab.fbow" -o orb_vocab.fbow', returncode=0)

In [24]:
# run the container
subprocess.run(
    f"docker exec {container_id} ./run_video_slam \
    -v ./orb_vocab.fbow \
    -c ./input/config.yaml \
    -m ./input/{video_name} \
    --map-db-out map.msg \
    --no-sleep \
    --eval-log-dir output \
    --start-timestamp 0",
    shell=True
)


[2025-01-04 04:45:26.706] [I] config file loaded: ./input/config.yaml
[2025-01-04 04:45:26.706] [I] 
original version of OpenVSLAM,
Copyright (C) 2019,
National Institute of Advanced Industrial Science and Technology (AIST)
All rights reserved.
stella_vslam (the changes after forking from OpenVSLAM),
Copyright (C) 2022, stella-cv, All rights reserved.

This is free software,
and you are welcome to redistribute it under certain conditions.
See the LICENSE file.

Camera:
  name: RICOH THETA S 960
  setup: monocular
  model: equirectangular
  fps: 30.0
  cols: 1280
  rows: 720
  color_order: RGB
  baseline: 0.1
Feature.max_num_keypoints: 2000
Feature.scale_factor: 1.2
Feature.num_levels: 8
Feature.ini_fast_threshold: 20
Feature.min_fast_threshold: 7
Feature.mask_rectangles:
  - [0.0, 1.0, 0.0, 0.1]
  - [0.0, 1.0, 0.84, 1.0]
  - [0.0, 0.2, 0.7, 1.0]
  - [0.8, 1.0, 0.7, 1.0]

[2025-01-04 04:45:26.706] [I] loading ORB vocabulary: ./orb_vocab.fbow
[2025-01-04 04:45:26.720] [I] load orb_params

CompletedProcess(args='docker exec 6ab7becb1dfb ./run_video_slam     -v ./orb_vocab.fbow     -c ./input/config.yaml     -m ./input/tokyo_0800_1010.mp4     --map-db-out map.msg     --no-sleep     --eval-log-dir output     --start-timestamp 0', returncode=0)

In [25]:
subprocess.run(
    f"docker cp {container_id}:/stella_vslam_examples/build/output .",
    shell=True
)

CompletedProcess(args='docker cp 6ab7becb1dfb:/stella_vslam_examples/build/output .', returncode=0)

In [ ]:
# visualize animation
# run animation.py